This is intended to be a simple notebook for going through analysis of the Perturb-CITE-seq data, and get a better sense of what the data looks like. 

Ressources:
1. [Paper](https://www.nature.com/articles/s41588-021-00779-1)
2. [Original code for reproducing the paper](https://github.com/klarman-cell-observatory/Perturb-CITE-seq) 
3. [Data release](https://singlecell.broadinstitute.org/single_cell/study/SCP1064/multi-modal-pooled-perturb-cite-seq-screens-in-patient-models-define-novel-mechanisms-of-cancer-immune-evasion)

# Step 0a: Download the data 

This (unfortunately) requires you to go through the following steps:
1. Go to the data release [page](https://singlecell.broadinstitute.org/single_cell/study/SCP1064/multi-modal-pooled-perturb-cite-seq-screens-in-patient-models-define-novel-mechanisms-of-cancer-immune-evasion)
2. Login / Create an account
3. Click on Download
4. Click on Bulk Download option
5. Copy **only** the variable *auth_code* from the URL and paste it **onto** the auth_code of the URL for the curl command below (each authorization code only works for 30 minute), and run the command (had to customize it).

It should take under two minute

In [3]:
!curl -k "https://singlecell.broadinstitute.org/single_cell/api/v1/bulk_download/generate_curl_config?accessions=SCP1064&auth_code=qD7R3EAB&directory=all" -o cfg.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  7747    0  7747    0     0   7602      0 --:--:--  0:00:01 --:--:--  7602


In [4]:
!curl -k -K cfg.txt && rm cfg.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  481M  100  481M    0     0   172M      0  0:00:02  0:00:02 --:--:--  172M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  811M  100  811M    0     0   144M      0  0:00:05  0:00:05 --:--:--  155M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  245M  100  245M    0     0  98.0M      0  0:00:02  0:00:02 --:--:-- 98.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3351M  100 3351M    0     0   163M      0  0:00:20  0:00:20 --:--:--  145M
  % Total    % Received % Xferd  Average Speed   Tim

## Step 1: Load the data onto scanpy

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.sparse as sp

In [2]:
data = sc.read_csv("../../perturb_cite_seq_data/SCP1064/other/RNA_expression.csv.gz").transpose()

# sparsify!
data_sp = sp.csr_matrix(data.X)

In [4]:
# get covariates
covariates = pd.read_csv("../../perturb_cite_seq_data/SCP1064/metadata/RNA_metadata.csv", index_col=0).iloc[1:, ]
data.obs = covariates

/scratch/4991657.sched-torque.pace.gatech.edu/ipykernel_132917/3568715894.py:2: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  covariates = pd.read_csv("../../perturb_cite_seq_data/SCP1064/metadata/RNA_metadata.csv", index_col=0).iloc[1:, ]


In [5]:
# correct dtypes
data.obs["MOI"] = data.obs["MOI"].astype(np.int32)
data.obs["UMI_count"] = data.obs["UMI_count"].astype(np.double)

/scratch/4991657.sched-torque.pace.gatech.edu/ipykernel_132917/2603284310.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.obs["MOI"] = data.obs["MOI"].astype(np.int32)
/scratch/4991657.sched-torque.pace.gatech.edu/ipykernel_132917/2603284310.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.obs["UMI_count"] = data.obs["UMI_count"].astype(np.double)


In [6]:
# de-normalize and round up
norm_factor =  data.obs["UMI_count"].values / 1.0e6
Z = sp.diags(norm_factor).dot(np.expm1(data_sp))
print(np.greater_equal(np.abs(Z.data - np.rint(Z.data)), 0.01).any())
Z.data = np.rint(Z.data)
data.X = Z

False


In [8]:
# read guide info
guide_info = pd.read_csv("../../perturb_cite_seq_data/SCP1064/documentation/all_sgRNA_assignments.txt", index_col=0)
guide_info = guide_info.replace(np.nan,'',regex=True)
data.obs["sgRNAs"] = guide_info["sgRNAs"].astype(str)

/scratch/4991657.sched-torque.pace.gatech.edu/ipykernel_132917/3963001815.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.obs["sgRNAs"] = guide_info["sgRNAs"].astype(str)


In [9]:
data.write_h5ad("../../perturb_cite_seq_data/SCP1064/other/adata.h5ad")

/storage/home/hcoda1/8/msethuraman7/p-ff14-0/miniconda3/envs/murali/lib/python3.10/site-packages/anndata/_core/anndata.py:1235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = c
/storage/home/hcoda1/8/msethuraman7/p-ff14-0/miniconda3/envs/murali/lib/python3.10/site-packages/anndata/_core/anndata.py:1235: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = c
/storage/home/hcoda1/8/msethuraman7/p-ff14-0/miniconda3/envs/murali/lib/python3.10/site-packages/anndata/_core/anndata.py:1235: SettingWithC

In [10]:
guide_info

,sgRNAs
Cell,
CELL_1,HLA-B_2
CELL_2,"NGFR_3,SERPINF1_3"
CELL_3,HLA-B_2
CELL_4,"NMRK1_3,S100A6_3"
CELL_5,
...,...
CELL_218327,"CTSO_3,PIK3IP1_3,VDAC2_2,WNT7A_1"
CELL_218328,"JAK2_3,SLC7A5P1_3"
CELL_218329,"S100A6_2,SAT1_2,ONE_NON-GENE_SITE_177"


In [11]:
covariates["condition"].value_counts()

IFNγ          87590
Co-culture    73114
Control       57627
Name: condition, dtype: int64

In [12]:
# Multiplicity of infection = number of guides per cell?
# Extended figure 2
covariates["MOI"].value_counts()

1     126966
2      45135
0      23028
3      14525
4       5053
5       1942
6        793
7        386
8        222
9        115
10        64
11        37
12        30
13        11
15         9
14         6
17         4
19         2
18         2
16         1
Name: MOI, dtype: int64

In [13]:
covariates["MOI"].mean()

1.3874850570922133

In [14]:
covariates["sgRNA"].value_counts()

IFNGR2_2       358
NO_SITE_47     333
NO_SITE_913    317
HLA-DRB5_2     315
NO_SITE_23     296
              ... 
PSMA7_1          2
DNAJC9_2         2
EIF2S3_3         1
UBC_2            1
TUBB_2           1
Name: sgRNA, Length: 818, dtype: int64